# Expert Systems

**Author: Brian van den Berg**

In this Python Notebook, I will be creating an expert system to demonstrate how it can be used in practise. First we will need to import the library 'experta', which is supported by Python 3.8.

In [5]:
import sys
import experta
from experta import *

# Print Python version
print("Python version:", sys.version)

# Print experta library version
print("experta version:", experta.__version__)

Python version: 3.8.10 (tags/v3.8.10:3d8993a, May  3 2021, 11:48:03) [MSC v.1928 64 bit (AMD64)]
experta version: 1.9.4


## Setting up Rules

To make an expert system work, we need to define the rules in the knowledge base that it has to follow. These rules are knowledge about an industry, set-up by experts within the industry. I however, an expert in the field of family visualization, have passed these rules on to the system when I created it using experta.

In [6]:
# Define a Fact class to represent individuals in the family tree
class Person(Fact):
    pass

# Define a Fact class to represent relations in the family tree
class Relation(Fact):
    pass

# Define a KnowledgeEngine class for the family tree expert system
class FamilyTree(KnowledgeEngine):
    """
    FamilyTree is an expert system that infers relations between people on the basis of individual people and their parents.
    The expert system has rules set-up that will use forward-chaining to look for rules that apply to a set of people.
    After creating a family tree, you use the run() function after which you can insert people using add_person().
    If you want to reset the family tree facts, you simply run reset().
    """

    # Rule to infer parent-child relations
    @Rule(Person(name=MATCH.x, parents=MATCH.parents))
    def parent_rule(self, x, parents):
        for parent in parents:
            self.declare(Relation(name=x, parent=parent))
            self.declare(Relation(name=parent, child=x))

    # Rule to infer sibling relations
    @Rule(Relation(name=MATCH.x, parent=MATCH.parent),
          Relation(name=MATCH.y, parent=MATCH.parent),
          TEST(lambda x, y: x != y))
    def sibling_rule(self, x, y):
        self.declare(Relation(name=x, sibling=y))

    # Rule to infer grandparent-grandchild relations
    @Rule(Relation(name=MATCH.x, parent=MATCH.parent),
          Relation(name=MATCH.parent, parent=MATCH.y))
    def grandparent_rule(self, x, y):
        self.declare(Relation(name=x, grandparent=y))
        self.declare(Relation(name=y, grandchild=x))

    # Rule to infer pibling-niephling relations
    @Rule(Relation(name=MATCH.x, grandparent=MATCH.grandparent),
          Relation(name=MATCH.y, parent=MATCH.grandparent))
    def pibling_rule(self, x, y):
        parents = self.get_parents(x)
        if y not in parents:
            self.declare(Relation(name=x, pibling=y))
            self.declare(Relation(name=y, niephling=x))

    # Rule to infer cousin relations
    @Rule(Relation(name=MATCH.x, pibling=MATCH.pibling),
          Relation(name=MATCH.y, parent=MATCH.pibling))
    def cousin_rule(self, x, y):
        self.declare(Relation(name=x, cousin=y))

    # Function to add a Person instance to the working memory
    def add_person(self, name, parents):
        self.declare(Person(name=name, parents=parents))

    # get_parents fetches every parent for a person, since we are dealing with an undetirmined number of parents
    def get_parents(self, person_name):
        parents = set()
        for i in self.facts:
            fact = self.facts[i]
            if fact.get('name') == person_name:
                if 'parent' in fact:
                    parents.add(fact['parent'])
        return parents

    # get_relations fetches every type of relation with another person in the family tree
    def get_relations(self, person_name):
        results = dict()

        # Go through every fact
        for i in self.facts:
            fact = self.facts[i]
            fact_person = fact.get('name')

            # Skip the facts about other people
            if fact_person != person_name:
                continue

            # Go through each key in the fact
            for key in fact:
                if key == '__factid__' or key == 'name' or key == 'parents':
                    continue

                # If the key is already in results, append otherwise initialize
                if key in results:
                    results[key].add(fact[key])
                else:
                    results[key] = set([fact[key]])

        return results
    
    # get_relation uses get_relations to find the specific relation between person1 and person2
    def get_relation(self, person1, person2):
        # Fetch the relations for person1
        r = self.get_relations(person1)

        # Go through every type of relation
        for relation_type in r.keys():
            person_set = r[relation_type]
            if person2 in person_set:
                return f'{person2} is the {relation_type} of {person1}'
        
        # Default case
        return f'{person1} does not have a direct relation with {person2}'

## Initializing the Expert System with Facts

When we initialize the family tree, we need to first create a family tree object. After we have done the first step, we need to reset the object with reset() and add instances of people to the family tree afterwards, which include their name and their parents. After we have set-up the people we wat to include in the family tree, we can simply use the run() function to start infering facts based on existing facts. The existing facts are the people we have added and the ones our system will infer are relations that can be derived from initial child-parents relations. To reset our system, we can simply run reset() and the database containing all of our facts will be emptied.

In [7]:
# Example of using the expert system with Person instances
family_tree = FamilyTree()
family_tree.reset()

# Create Person instances and add them to the expert system
family_tree.add_person("Ethan", ["Noah", "Ava"])
family_tree.add_person("Olivia", ["Noah", "Ava"])
family_tree.add_person("Mason", ["Noah", "Ava"])
family_tree.add_person("Noah", ["Logan", "Sophia"])
family_tree.add_person("Ava", ["Caleb", "Isabella"])
family_tree.add_person("Logan", ["Jackson", "Grace"])
family_tree.add_person("Sophia", ["Liam", "Emily"])
family_tree.add_person("Caleb", ["Owen", "Aria"])
family_tree.add_person("Isabella", ["Carter", "Lily"])
family_tree.add_person("Amelia", ["Logan", "Sophia"])
family_tree.add_person("Dylan", ["Caleb", "Isabella"])
family_tree.add_person("Stella", ["Dylan", "Scarlett", "Harper"])
family_tree.add_person("Sebastian", ["Dylan", "Scarlett", "Harper"])
family_tree.add_person("Penelope", ["Dylan", "Scarlett", "Harper"])
family_tree.add_person("Lucas", ["Amelia", "Gabriel"])

# Run the expert system
family_tree.run()

# Print the steps the inference engine took to infer facts
print(family_tree.facts)

<f-0>: InitialFact()
<f-1>: Person(name='Ethan', parents=frozenlist(['Noah', 'Ava']))
<f-2>: Person(name='Olivia', parents=frozenlist(['Noah', 'Ava']))
<f-3>: Person(name='Mason', parents=frozenlist(['Noah', 'Ava']))
<f-4>: Person(name='Noah', parents=frozenlist(['Logan', 'Sophia']))
<f-5>: Person(name='Ava', parents=frozenlist(['Caleb', 'Isabella']))
<f-6>: Person(name='Logan', parents=frozenlist(['Jackson', 'Grace']))
<f-7>: Person(name='Sophia', parents=frozenlist(['Liam', 'Emily']))
<f-8>: Person(name='Caleb', parents=frozenlist(['Owen', 'Aria']))
<f-9>: Person(name='Isabella', parents=frozenlist(['Carter', 'Lily']))
<f-10>: Person(name='Amelia', parents=frozenlist(['Logan', 'Sophia']))
<f-11>: Person(name='Dylan', parents=frozenlist(['Caleb', 'Isabella']))
<f-12>: Person(name='Stella', parents=frozenlist(['Dylan', 'Scarlett', 'Harper']))
<f-13>: Person(name='Sebastian', parents=frozenlist(['Dylan', 'Scarlett', 'Harper']))
<f-14>: Person(name='Penelope', parents=frozenlist(['Dylan'

## Usage of the System

Currently there are only two functions in my expert system to fetch relations between people. The first function is "get_relations(person: str) -> dict[str, set]" which fetches every relationregarding a person with O(n) complexity (n being the number of facts in the expert system). The second function is "get_relation(person1: str, person2: str) -> str" which fetches a sentence describing specifically how two people are related with eachother. The time complexity of this second function is also O(n), but I believe this could be improved on by applying a smart lookup instead of a for loop. This would require the system to be set-up to support smart lookups in facts for names.

In [8]:
# Usage example
person = "Ethan"

# Printing the formatted output
print(f'Relations for {person}:')
print(f'- {family_tree.get_relations(person)}')
print()

# Asking for a specific relationship
print("Relation between Ethan and Sophia:")
print(f'- {family_tree.get_relation("Ethan", "Sophia")}')

Relations for Ethan:
- {'parent': {'Noah', 'Ava'}, 'sibling': {'Mason', 'Olivia'}, 'grandparent': {'Sophia', 'Logan', 'Isabella', 'Caleb'}, 'pibling': {'Amelia', 'Dylan'}, 'cousin': {'Penelope', 'Sebastian', 'Lucas', 'Stella'}}

Relation between Ethan and Sophia:
- Sophia is the grandparent of Ethan


## Future Proof

In order to make the system future proof, I used a variable amount of parents and gender neutral relation terms. This was done to abstractify the entire concept of family to its core component. The important thing is that the family tree represents how a family is connected to eachother and when this system would be furthur developed on, you could replace names with ID's and assign more information to the people in a database. Gender would for example be defined in a seperate dataset to divide functionality from big data.

On this scale, it was not required to abstractify furthur than this, because duplicate names is not an issue yet. When one would implement a system like this in real applications, it would be required to fix the issue of duplicate names.